In [1]:
import pandas as pd
pd.set_option('display.max_columns', 500)
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from utils import getXY
import numpy as np
import keras

Using TensorFlow backend.


In [2]:
cleandf = pd.read_csv('cleanData.csv')
cleandf = cleandf[cleandf.roles == 'Student']
cleandf.drop(['cc_by_ip', 'roles'], axis=1, inplace=True)

In [3]:
cleandf.head()

,course_id,LoE,YoB,gender,nevents,grade,ndays_act,nchapters,nforum_posts,avg_dt
0,PH,p,1971,m,1168,0.70,12,5,1,19.838340
1,ER,b,1986,m,8642,0.61,53,32,2,16.891964
2,ER,b,1978,m,5923,0.62,31,32,9,12.206571
3,ER,other,1973,m,6134,0.62,80,32,3,17.667405
4,ER,hs,1989,f,4788,0.62,42,32,2,22.744499


In [4]:
def change_to_classes(x):
    if x < 0.33:
        return 0
    elif x < 0.66:
        return 1
    else:
        return 2
cleandf['grade_class'] = cleandf['grade'].apply(lambda x: change_to_classes(x))
cleandf.drop(['grade'], axis=1, inplace=True)

In [5]:
#params
response = ['grade_class']
clip_cols = ['YoB']
clip_vals = [[2013-80, 2018-10], [0, 1]]
dummy_cols = ['LoE', 'gender', 'course_id']
quant_cols = ['YoB', 'nevents', 'ndays_act', 'nchapters', 'nforum_posts', 'avg_dt']
X, Y = getXY(cleandf, response, clip_cols, clip_vals, dummy_cols, quant_cols)

#train test split
trainX, testX, trainY, testY = train_test_split(X, Y, test_size = 0.20, random_state = 208)

/usr/share/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


Normalizaton complete.
Dummify complete.


In [6]:
trainX.shape, trainY.shape, testX.shape, testY.shape

((146473, 51), (146473, 1), (36619, 51), (36619, 1))

In [7]:
trainX.head()

,YoB,nevents,ndays_act,nchapters,nforum_posts,avg_dt,LoE_a,LoE_b,LoE_el,LoE_hs,LoE_jhs,LoE_m,LoE_none,LoE_other,LoE_p,LoE_p_oth,LoE_p_se,gender_f,gender_m,gender_o,course_id_1368,course_id_AI,course_id_AT,course_id_AmPoX,course_id_BUS,course_id_CB,course_id_CHEM,course_id_CHN,course_id_CalcAPL,course_id_EMC,course_id_ENGSCI,course_id_ER,course_id_FC,course_id_GSD,course_id_GSE,course_id_HDS,course_id_HHP,course_id_HKS,course_id_HLS,course_id_HSPH,course_id_HUM,course_id_Hum,course_id_MCB,course_id_MUS,course_id_PH,course_id_SOC,course_id_SPU,course_id_SW,course_id_USW,course_id_Urban,course_id_VJx
3321,0.760000,0.000695,0.004617,0.200000,0.000531,0.055816,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
73051,0.680000,0.000248,0.003412,0.066667,0.000000,0.062323,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
66659,0.480000,0.000673,0.001004,0.311111,0.003449,0.078585,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
102507,0.706667,0.002651,0.006022,0.177778,0.001061,0.039079,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
59348,0.760000,0.000053,0.002007,0.044444,0.000265,0.053437,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [8]:
print('baseline accuracy - predicting most common class' , trainY.grade_class.value_counts()[0]/len(Y))

baseline accuracy - predicting most common class 0.4508116138334826


In [9]:
trainY.grade_class.value_counts()

0    82540
2    49920
1    14013
Name: grade_class, dtype: int64

### Random Forest

In [10]:
rfc = RandomForestClassifier(n_estimators=100, min_samples_split = 10)

In [11]:
# Y = np.array(Y).reshape(-1,)

In [12]:
rfc.fit(trainX, np.array(trainY).reshape(-1,))

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=10,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [13]:
rfc.score(trainX, np.array(trainY).reshape(-1,))

0.9235558771923836

In [14]:
rfc.score(testX, np.array(testY).reshape(-1,))

0.855430241131653

### Neural Net

In [11]:
nn_trainY = keras.utils.to_categorical(trainY)
nn_testY = keras.utils.to_categorical(testY)

In [12]:
nn_testY

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.]], dtype=float32)

In [25]:
input_dim = trainX.shape[1]

# build the model

model_nn = keras.Sequential()

model_nn.add(keras.layers.Dense(50, input_dim = input_dim, 
                      activation = 'relu'))

model_nn.add(keras.layers.Dense(25, 
                      activation = 'relu'))

model_nn.add(keras.layers.Dense(3, 
                       activation='sigmoid')) 

sgd = keras.optimizers.SGD(lr=0.01)

# compile the model
model_nn.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['acc'])


In [26]:
model_nn.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_13 (Dense)             (None, 50)                2600      
_________________________________________________________________
dense_14 (Dense)             (None, 25)                1275      
_________________________________________________________________
dense_15 (Dense)             (None, 3)                 78        
Total params: 3,953
Trainable params: 3,953
Non-trainable params: 0
_________________________________________________________________


In [27]:
model_nn_history = model_nn.fit(x = trainX, y = nn_trainY, 
                                batch_size=256, 
                                epochs = 10, 
                                verbose = 1,
                                validation_data = (testX, nn_testY))

Train on 146473 samples, validate on 36619 samples
Epoch 1/10
146473/146473 [==============================] - 2s 13us/step - loss: 0.9936 - acc: 0.5188 - val_loss: 0.9179 - val_acc: 0.5556
Epoch 2/10
146473/146473 [==============================] - 2s 12us/step - loss: 0.9053 - acc: 0.5635 - val_loss: 0.9004 - val_acc: 0.5556
Epoch 3/10
146473/146473 [==============================] - 2s 12us/step - loss: 0.8889 - acc: 0.5637 - val_loss: 0.8817 - val_acc: 0.5572
Epoch 4/10
146473/146473 [==============================] - 2s 12us/step - loss: 0.8629 - acc: 0.5989 - val_loss: 0.8461 - val_acc: 0.6246
Epoch 5/10
146473/146473 [==============================] - 2s 12us/step - loss: 0.8129 - acc: 0.6506 - val_loss: 0.7798 - val_acc: 0.6743
Epoch 6/10
146473/146473 [==============================] - 2s 12us/step - loss: 0.7351 - acc: 0.7045 - val_loss: 0.6924 - val_acc: 0.7272
Epoch 7/10
146473/146473 [==============================] - 2s 12us/step - loss: 0.6509 - acc: 0.7531 - val_loss: 0

In [27]:
# trainX.to_csv('trainX.csv')
# testX.to_csv('testX.csv')
# trainY.to_csv('trainY.csv')
# testY.to_csv('testY.csv')

In [17]:
trainY.grade_class.unique()

array([2, 1, 0])

In [28]:
model_nn_history.history

{'val_loss': [0.5242817539507787, 0.505711113183534],
 'val_acc': [0.729985369860291, 0.7490465097973102],
 'loss': [0.5296937704503817, 0.5131988915805871],
 'acc': [0.7211431679973029, 0.7415132485454473]}

In [21]:
import json
# hist = model.fit(X_train, y_train, epochs=5, batch_size=batch_size,validation_split=0.1)
with open('edx_model_hist.json', 'w') as f:
    json.dump(model_nn_history.history, f)

In [22]:
with open('edx_model_hist.json', 'r') as f:
    check = json.loads(f.read())

In [25]:
check

{'val_loss': [0.5397967763088236],
 'val_acc': [0.7039788288129841],
 'loss': [0.5764122510158524],
 'acc': [0.705556679490453]}

In [31]:
146473//256 * 256 - 146473

-41

In [ ]:
import tensorflow as tf

---